In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../../')
import velovae as vv

In [2]:
dataset = 'IO_EU'
adata = anndata.read_h5ad(f'data/{dataset}_pp.h5ad')
adata.obs['clusters'] = adata.obs['cell_type'].to_numpy()
import pandas as pd
adata.var.index = pd.Index(adata.var['gene'])

In [ ]:
vv.preprocess(adata, n_gene=2000, min_shared_counts=20)

In [5]:
model_path_base = f'checkpoints/{dataset}'
figure_path_base = f'figures/{dataset}'
data_path = f'data/velovae/discrete/{dataset}'
gene_plot = ['Lgr5', 'Apoa1', 'Dgat1', 'Gsta4']

# Discrete VeloVAE

In [6]:
figure_path = f'{figure_path_base}/VeloVAE'
model_path = f'{model_path_base}/VeloVAE'

torch.manual_seed(2022)
np.random.seed(2022)
dvae = vv.DVAE(adata, 
               tmax=20, 
               dim_z=5, 
               device='cuda:0',
               init_method='steady')

dvae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

dvae.save_model(model_path, 'encoder', 'decoder')
dvae.save_anndata(adata, 'dvae', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Initialization using the steady-state and dynamical models.


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
Library scale (U): Max=6.55, Min=0.09, Mean=0.98
Library scale (S): Max=10.58, Min=0.17, Mean=1.02
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 21, test iteration: 40
*********       Stage 1: Early Stop Triggered at epoch 223.       *********
*********                      Stage  2                       *********


  0%|          | 0/3829 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.005
Average Set Size: 195
*********       Stage 2: Early Stop Triggered at epoch 254.       *********
*********              Finished. Total Time =   0 h :  1 m : 46 s             *********
Final: Train ELBO = -1877.377,           Test ELBO = -1921.024
       Training MSE = 1.386, Test MSE = 1.524


# Discrete Full VB

In [7]:
figure_path = f'{figure_path_base}/DFullVB'
model_path = f'{model_path_base}/DFullVB'

torch.manual_seed(2022)
np.random.seed(2022)
full_vb = vv.VAEFullVB(adata, 
                       tmax=20, 
                       dim_z=5, 
                       device='cuda:0',
                       init_method='steady')

full_vb.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

full_vb.save_model(model_path, 'encoder', 'decoder')
full_vb.save_anndata(adata, 'dfullvb', data_path, file_name=f'{dataset}.h5ad')

Initialization using the steady-state and dynamical models.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 21, test iteration: 40
*********       Stage 1: Early Stop Triggered at epoch 490.       *********
*********                      Stage  2                       *********
Cell-wise KNN Estimation.


  0%|          | 0/3829 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.090
Average Set Size: 167
Finished. Actual Time:   0 h :  0 m :  3 s
*********       Stage 2: Early Stop Triggered at epoch 900.       *********
*********              Finished. Total Time =   0 h :  5 m : 19 s             *********
Final: Train ELBO = 3433.425,           Test ELBO = 3356.509


# Evaluation

In [ ]:
cluster_edges = [('Stem cells', 'TA cells'), ('Stem cells', 'Globet Cells')]
vv.post_analysis(adata,
                 'eval',
                 ['Discrete VeloVAE','Discrete FullVB'],
                 ['dvae','dfullvb'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges)